### （模型融合） - 对Task3调优后的模型, 进行模型融合。

例如, 用目前评分最高的模型作为基准模型, 和其他模型进行stacking模型融合, 得到最终模型和评分。

首先选几个模型作为第一层模型进行5折交叉验证。

例如：分别选用LR、SVM和决策树作为第一层模型Model1，分别实施以下步骤：

1）把train分train1~train5，共5份，用其中4份预测剩下的那份，同时预测test数据，这样的过程做5次，生成5份train（原train样本数/5）数据和5份test数据。

2）把5份预测的train数据纵向叠起来（即为原来的整个train，只是顺序不同），把test预测的结果做平均。

将三个模型得到的train并列合并成一个3列的矩阵作为training-data，得到的test并列合并成一个3列的矩阵作为testing-data。

让下一层的模型，基于他们进一步训练。

In [1]:
import pandas as pd
import pickle

# 数据集预览
data = pd.read_csv('data_2.csv')
# 去重
data.drop_duplicates(inplace=True)
print(data.shape)
data.head()
# 载入特征
with open('feature.pkl', 'rb') as f:
    X = pickle.load(f)

# 观测正负样本是否均衡
y = data.status
y.value_counts()


(4754, 90)


0    3561
1    1193
Name: status, dtype: int64

In [2]:
# 划分训练集测试集
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=2333)

# 特征归一化
std = StandardScaler()
X_train = std.fit_transform(X_train.values)
X_test = std.transform(X_test.values)


#### 模型评估

In [3]:
from sklearn.metrics import accuracy_score, roc_auc_score

def model_metrics(clf, X_train, X_test, y_train, y_test):
    # 预测
    y_train_pred = clf.predict(X_train)
    y_test_pred = clf.predict(X_test)
    
    y_train_proba = clf.predict_proba(X_train)[:,1]
    y_test_proba = clf.predict_proba(X_test)[:,1]
    
    # 准确率
    print('[准确率]', end = ' ')
    print('训练集：', '%.4f'%accuracy_score(y_train, y_train_pred), end = ' ')
    print('测试集：', '%.4f'%accuracy_score(y_test, y_test_pred))
    
    # auc取值：用roc_auc_score或auc
    print('[auc值]', end = ' ')
    print('训练集：', '%.4f'%roc_auc_score(y_train, y_train_proba), end = ' ')
    print('测试集：', '%.4f'%roc_auc_score(y_test, y_test_proba))


#### 模型选择


In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from xgboost.sklearn import XGBClassifier
from lightgbm.sklearn import LGBMClassifier
from mlxtend.classifier import StackingClassifier

# 模型调优后得到的参数
lr = LogisticRegression(C = 0.1, penalty = 'l1')
svm_linear = svm.SVC(C = 0.01, kernel = 'linear', probability=True)
svm_poly =  svm.SVC(C = 0.01, kernel = 'poly', probability=True)
svm_rbf =  svm.SVC(gamma = 0.01, C =0.01 , probability=True)
svm_sigmoid =  svm.SVC(C = 0.01, kernel = 'sigmoid',probability=True)
dt = DecisionTreeClassifier(max_depth=11,min_samples_split=550,min_samples_leaf=80,max_features=19)
xgb = XGBClassifier(learning_rate =0.01, n_estimators=180, max_depth=3, min_child_weight=5, 
                    gamma=0.4, subsample=0.5, colsample_bytree=0.9, reg_alpha=1, 
                    objective= 'binary:logistic', nthread=4,scale_pos_weight=1, seed=27)
lgb = LGBMClassifier(learning_rate =0.1, n_estimators=60, max_depth=3, min_child_weight=11, 
                    gamma=0.1, subsample=0.5, colsample_bytree=0.8, reg_alpha=1e-5, 
                    nthread=4,scale_pos_weight=1, seed=27)


#### 2.1 调包实现

使用4种SVM、决策树、XGBoost和LightGBM作为初级分类器，LR作为次级分类器。

1）将初级分类器产生的类别标签作为新特征

In [5]:
import warnings
warnings.filterwarnings("ignore")

sclf = StackingClassifier(classifiers=[svm_linear, svm_poly, svm_rbf, 
                                       svm_sigmoid, dt, xgb, lgb], meta_classifier=lr)
sclf.fit(X_train, y_train.values)
model_metrics(sclf, X_train, X_test, y_train, y_test)


[准确率] 训练集： 0.8260 测试集： 0.8045
[auc值] 训练集： 0.6933 测试集： 0.6599


2）将初级分类器产生的输出类概率作为新特征

对输出概率use_probas=True，有两种不同的处理方式。

假设有2个初级分类器和3个类别输出概率：p1=[0.2,0.5,0.3],p2=[0.3,0.4,0.4].

如果average_probas=True，则对分类器的结果求平均，得到：p=[0.25,0.45,0.35]

[推荐]如果average_probas=False，则分类器的所有结果都保留作为新的特征：p=[0.2,0.5,0.3,0.3,0.4,0.4]


In [6]:
sclf = StackingClassifier(classifiers=[svm_linear, svm_poly, svm_rbf, svm_sigmoid, dt, xgb, lgb],  meta_classifier=lr, use_probas=True,average_probas=False)
sclf.fit(X_train, y_train.values)
model_metrics(sclf, X_train, X_test, y_train, y_test)


[准确率] 训练集： 0.8374 测试集： 0.8024
[auc值] 训练集： 0.8776 测试集： 0.7978
